## 1. Introduction 
The main goal of this data analysis is to optimize marketing expenses of the given online retail store. The marketing department
requested data analyst to find where they need to invest more for marketing advertisement so that store generates higher revenue. Data collection is done on products, sales, and marketing expenses from June 2017 through May 2018. The metric values of products, sales, and marketing is used to analyze which platforms/sources need to have more marketing investment.



The answers of the following questions will be reported and then the final conclusion will be made accordingly.
<br>
How people use the product?
<br>
When they start to buy?
<br>
How much money each customer brings?
<br>
When they pay off?

**Product**<br>
How many people use it every day, week, and month?<br>
How many sessions are there per day? (One user might have more than one session.)<br>
What is the length of each session?<br>
What's the user retention rate?<br>

**Sales**<br>
When do people start buying? (In KPI analysis, we're usually interested in knowing the time that elapses between registration and conversion — when the user becomes a customer. For example, if registration and the first purchase occur on the same day, the user might fall into category Conversion 0d. If the first purchase happens the next day, it will be Conversion 1d. You can use any approach that lets you compare the conversions of different cohorts, so that you can determine which cohort, or marketing channel, is most effective.)<br>
How many orders do they make during a given period of time?<br>
What is the average purchase size?<br>
How much money do they bring? (LTV)<br>

**Marketing**<br>
How much money was spent? Overall, per source and over time.<br>
How much did customer acquisition from each of the sources cost?<br>
How worthwhile where the investments? (ROI)

## 2. Import files and General Information

In [6]:
# loading the libraries
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

In [7]:
# extract files
try:
    visit_df=pd.read_csv("visits_log_us.csv")
    order_df=pd.read_csv("orders_log_us.csv")
    marketing_df=pd.read_csv("costs_us.csv")
except:
    visit_df=pd.read_csv("/datasets/visits_log_us.csv")
    order_df=pd.read_csv("/datasets/orders_log_us.csv")
    marketing_df=pd.read_csv("/datasets/costs_us.csv")
    

**Data Describtion**<br>
The visits table (server logs with data on website visits):<br>
Uid — user's unique identifier<br>
Device — user's device<br>
Start Ts — session start date and time<br>
End Ts — session end date and time<br>
Source Id — identifier of the ad source the user came from<br>

All dates in this table are in YYYY-MM-DD format.<br>
The orders table (data on orders):<br>
Uid — unique identifier of the user making an order<br>
Buy Ts — order date and time<br>
Revenue — Yandex.Afisha's revenue from the order

The costs table (data on marketing expenses):<br>
source_id — ad source identifier<br>
dt — date<br>
costs — expenses on this ad source on this day

In [8]:
# visit files sample
visit_df.sample()

,Device,End Ts,Source Id,Start Ts,Uid
154928,desktop,2017-10-06 14:12:00,10,2017-10-06 13:59:00,5652580742521997619


In [9]:
# order files sample
order_df.head(5)

,Buy Ts,Revenue,Uid
0,2017-06-01 00:10:00,17.00,10329302124590727494
1,2017-06-01 00:25:00,0.55,11627257723692907447
2,2017-06-01 00:27:00,0.37,17903680561304213844
3,2017-06-01 00:29:00,0.55,16109239769442553005
4,2017-06-01 07:58:00,0.37,14200605875248379450


In [10]:
# cost of marketing samples
marketing_df.head(5)

,source_id,dt,costs
0,1,2017-06-01,75.20
1,1,2017-06-02,62.25
2,1,2017-06-03,36.53
3,1,2017-06-04,55.00
4,1,2017-06-05,57.08


In [11]:
# info of datasets
visit_df.info()
print()
order_df.info()
print()
marketing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359400 entries, 0 to 359399
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Device     359400 non-null  object
 1   End Ts     359400 non-null  object
 2   Source Id  359400 non-null  int64 
 3   Start Ts   359400 non-null  object
 4   Uid        359400 non-null  uint64
dtypes: int64(1), object(3), uint64(1)
memory usage: 13.7+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50415 entries, 0 to 50414
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Buy Ts   50415 non-null  object 
 1   Revenue  50415 non-null  float64
 2   Uid      50415 non-null  uint64 
dtypes: float64(1), object(1), uint64(1)
memory usage: 1.2+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2542 entries, 0 to 2541
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  

In [12]:
# change space in column with underscore
visit_df.columns=visit_df.columns.str.replace(" ","_").str.lower()
order_df.columns=order_df.columns.str.replace(" ","_").str.lower()
marketing_df.columns=marketing_df.columns.str.replace(" ","_").str.lower()

In [13]:
# check for column names replacement
visit_df.columns

Index(['device', 'end_ts', 'source_id', 'start_ts', 'uid'], dtype='object')

In [14]:
# missing values
visit_df.isnull().sum()

device       0
end_ts       0
source_id    0
start_ts     0
uid          0
dtype: int64

In [15]:
# missing values
order_df.isnull().sum()

buy_ts     0
revenue    0
uid        0
dtype: int64

In [16]:
# missing values
marketing_df.isnull().sum()

source_id    0
dt           0
costs        0
dtype: int64

In [17]:
# duplicate values
visit_df.duplicated().sum()

0

In [18]:
# duplicate values
order_df.duplicated().sum()

0

In [19]:
# duplicate values
marketing_df.duplicated().sum()

0

In [20]:
# duplicate values
marketing_df.duplicated().sum()

0

In [21]:
marketing_df.columns

Index(['source_id', 'dt', 'costs'], dtype='object')

In [22]:
# change time column to time data type
visit_df["end_ts"]=pd.to_datetime(visit_df["end_ts"])
visit_df["start_ts"]=pd.to_datetime(visit_df["start_ts"])
order_df["buy_ts"]=pd.to_datetime(order_df["buy_ts"])
marketing_df["dt_ts"]=pd.to_datetime(marketing_df["dt"])


In [23]:
# check for columns data type
display(visit_df.dtypes)
display(order_df.dtypes)
display(marketing_df.dtypes)

device               object
end_ts       datetime64[ns]
source_id             int64
start_ts     datetime64[ns]
uid                  uint64
dtype: object

buy_ts     datetime64[ns]
revenue           float64
uid                uint64
dtype: object

source_id             int64
dt                   object
costs               float64
dt_ts        datetime64[ns]
dtype: object